In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
import os

In [2]:
from tensorflow.keras.applications import ResNet50

In [3]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import load_model

In [4]:
import numpy as np
from sklearn import metrics

In [5]:
from experiments.data_gen import PascalVOCDataGenerator

Using TensorFlow backend.


In [6]:
from model.callbacks.metric_callbacks import MAPCallback
from model.callbacks.save_callback import SaveModel

In [22]:
from model.losses import BCE, PartialBCE

In [8]:
from sklearn.metrics import average_precision_score

In [54]:
prop=60

## model

In [ ]:
# From http://cedric.cnam.fr/vertigo/Cours/ml2/tpDeepLearning5.html

In [ ]:
# Load ResNet50 architecture & its weights
# imagenet_model = ResNet50(include_top=True, weights='imagenet')
# model.layers.pop()
# Modify top layers
# x = model.layers[-1].output
# x = Dense(data_generator_train.nb_classes, activation='sigmoid', name='predictions')(x)
# model = Model(inputs=model.input, outputs=x)

In [ ]:
# imagenet only model (no finetuning)
model = ResNet50(include_top=True, weights='imagenet')

In [ ]:
# model.layers.pop().pop()
model = Model(inputs=model.input,outputs=model.layers[-2].output)

In [ ]:
model.summary()

In [ ]:
lr = 0.1
model.compile(loss='binary_crossentropy', optimizer=SGD(lr, momentum=0.9), metrics=['binary_accuracy'])

## feature extraction

In [ ]:
data_dir = '/share/DEEPLEARNING/datasets/pascalvoc/VOCdevkit/VOC2007/'
data_generator_train = PascalVOCDataGenerator('trainval', data_dir, prop=prop)

In [ ]:
data_generator_train = PascalVOCDataGenerator('trainval', data_dir, force_old=True)

In [ ]:
batch_size = 32
generator = data_generator_train.flow(batch_size=batch_size)

In [ ]:
# Initilisation des matrices contenant les Deep Features et les labels
X_train = np.zeros((len(data_generator_train.images_ids_in_subset),2048))
Y_train = np.zeros((len(data_generator_train.images_ids_in_subset),20))

# Calcul du nombre e batchs
nb_batches = int(len(data_generator_train.images_ids_in_subset) / batch_size) + 1

for i in range(nb_batches):
    # Pour chaque batch, on extrait les images d'entrée X et les labels y
    X, y = next(generator)
    # On récupère les Deep Feature par appel à predict
    y_pred = model.predict(X)
    X_train[i*batch_size:(i+1)*batch_size,:] = y_pred
    Y_train[i*batch_size:(i+1)*batch_size,:] = y

In [ ]:
# same for test
data_generator_test = PascalVOCDataGenerator('test', data_dir)
generator_test = data_generator_test.flow(batch_size=batch_size)

X_test = np.zeros((len(data_generator_test.images_ids_in_subset),2048))
Y_test = np.zeros((len(data_generator_test.images_ids_in_subset),20))

nb_batches = int(len(data_generator_test.images_ids_in_subset) / batch_size) + 1

for i in range(nb_batches):
    X, y = next(generator_test)
    y_pred = model.predict(X)
    X_test[i*batch_size:(i+1)*batch_size,:] = y_pred
    Y_test[i*batch_size:(i+1)*batch_size,:] = y

In [ ]:
outfile = 'DF_ResNet50_VOC2007_test28'
np.savez(outfile, X_train=X_train, Y_train=Y_train,X_test=X_test, Y_test=Y_test)

## Train classifier only

In [ ]:
outfile = 'DF_ResNet50_VOC2007_test28.npz'
learning_rate = 0.1
nb_epoch = 20
batch_size = 32

npzfile = np.load(outfile)

X_train = npzfile['X_train']
Y_train = npzfile['Y_train']

X_test = npzfile['X_test']
Y_test = npzfile['Y_test']

print("data \n X_train=", X_train.shape, "Y_train=", Y_train.shape, " X_test=", X_test.shape, "Y_train=", Y_test.shape)

In [ ]:
model = Sequential()
model.add(Dense(20, input_dim=2048, name='fc1', activation='sigmoid'))
model.summary()


sgd = SGD(learning_rate)
model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['binary_accuracy'])

model.fit(X_train, Y_train,batch_size=batch_size, epochs=nb_epoch,verbose=1)

In [ ]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("%s TEST: %.2f%%" % (model.metrics_names[0], scores[0]*100))
print("%s TEST: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)
AP_train = np.zeros(20)
AP_test = np.zeros(20)

for c in range(20):
    AP_train[c] = average_precision_score(Y_train[:, c], y_pred_train[:, c])
    AP_test[c] = average_precision_score(Y_test[:, c], y_pred_test[:, c])

print("MAP TRAIN =%.2f", AP_train.mean()*100)
print("MAP TEST =%.2f", AP_test.mean()*100)

## Eval

In [ ]:
data_generator_test = PascalVOCDataGenerator('test', data_dir)
len(data_generator_test.images_ids_in_subset)

In [ ]:
batch_size = len(data_generator_test.images_ids_in_subset)
generator_test = data_generator_test.flow(batch_size=batch_size)

In [ ]:
X_test, Y_test = next(generator_test)

In [ ]:
print(X_test.shape, Y_test.shape)

In [ ]:
def eval_fn(model, X_test, Y_test):
    y_pred_test = model.predict(X_test)
    #y_pred_train = model.predict(X_train)

    #AP_train = np.zeros(20)
    AP_test = np.zeros(20)
    for c in range(20):
        #AP_train[c] = average_precision_score(Y_train[:, c], y_pred_train[:, c])
        AP_test[c] = metrics.average_precision_score(Y_test[:, c], y_pred_test[:, c])

    #print "MAP TRAIN =", AP_train.mean()*100
    print("MAP TEST =", AP_test.mean()*100)
    print(AP_test)

In [ ]:
eval_fn(model, X_test, Y_test)

In [ ]:
print(AP_test)

## Finetune

In [55]:
data_dir = '/share/DEEPLEARNING/datasets/pascalvoc/VOCdevkit/VOC2007/'
data_generator_train = PascalVOCDataGenerator('trainval', data_dir, prop=prop)

loading dataset from /share/DEEPLEARNING/datasets/pascalvoc/VOCdevkit/VOC2007/Annotations/annotations_multilabel_trainval_partial_60_1.csv


In [49]:
# vanilla from RCP209
model = ResNet50(include_top=True, weights='imagenet')
model.layers.pop()
# Modify top layers
x = model.layers[-1].output
x = Dense(data_generator_train.nb_classes, activation='sigmoid', name='predictions')(x)
model = Model(inputs=model.input, outputs=x)

In [56]:
# dropping the Dense(1000)
model = ResNet50(include_top=True, weights='imagenet')
x = model.layers[-2].output
x = Dense(data_generator_train.nb_classes, activation='sigmoid', name='predictions')(x)
model = Model(inputs=model.input, outputs=x)

In [ ]:
# Laura way
# Load ResNet50 architecture & its weights
input_shape = (224, 224, 3)
resnet = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)

inp = Input(shape=input_shape, name='image_input')
x = resnet(inp)
x = Flatten()(x)
output = Dense(data_generator_train.nb_classes, activation='sigmoid')(x)
model = Model(inputs=inp, outputs=output)

In [ ]:
model.summary()

In [57]:
lr = 0.1
# model.compile(loss='binary_crossentropy', optimizer=SGD(lr=lr), metrics=['binary_accuracy'])
# loss = BCE()
loss = PartialBCE(prop / 100)
model.compile(loss=loss, optimizer=SGD(lr=lr), metrics=['binary_accuracy'])

In [15]:
# test data
data_generator_test = PascalVOCDataGenerator('test', data_dir)
len(data_generator_test.images_ids_in_subset)
batch_size = len(data_generator_test.images_ids_in_subset)
generator_test = data_generator_test.flow(batch_size=batch_size)
X_test, Y_test = next(generator_test)
print(X_test.shape, Y_test.shape)

loading dataset from /share/DEEPLEARNING/datasets/pascalvoc/VOCdevkit/VOC2007/Annotations/annotations_multilabel_test.csv
(4952, 224, 224, 3) (4952, 20)


In [58]:
# callbacks
exp_folder = '/home/caleml/partial_experiments/exp_20190722_1416_TESTNB'
os.makedirs(exp_folder, exist_ok=True)

cb_list = list()
cb_list.append(SaveModel(exp_folder, prop))

map_cb = MAPCallback(X_test, Y_test, exp_folder, prop)
cb_list.append(map_cb)

In [59]:
batch_size=32
nb_epochs=25

steps_per_epoch_train = int(len(data_generator_train.id_to_label) / batch_size) + 1
model.fit_generator(data_generator_train.flow(batch_size=batch_size),
                    steps_per_epoch=steps_per_epoch_train,
                    epochs=nb_epochs,
                    callbacks=cb_list,
                    verbose=1)

Epoch 1/25
156/157 [============================>.] - ETA: 0s - loss: 0.3343 - binary_accuracy: 0.4050
Trying to save model @epoch=001 to /home/caleml/partial_experiments/exp_20190722_1416_TESTNB/model_60_001.h5
Couldn't save model, saving weights instead at /home/caleml/partial_experiments/exp_20190722_1416_TESTNB/weights_60_001.h5
type true <class 'numpy.ndarray'>, type pred <class 'numpy.ndarray'>
ap scores type <class 'numpy.ndarray'>
interval evaluation - epoch: 0 - mAP score: 0.787593
157/157 [==============================] - 266s 2s/step - loss: 0.3344 - binary_accuracy: 0.4050
Epoch 2/25
156/157 [============================>.] - ETA: 0s - loss: 0.1226 - binary_accuracy: 0.4150
Trying to save model @epoch=002 to /home/caleml/partial_experiments/exp_20190722_1416_TESTNB/model_60_002.h5
Couldn't save model, saving weights instead at /home/caleml/partial_experiments/exp_20190722_1416_TESTNB/weights_60_002.h5
type true <class 'numpy.ndarray'>, type pred <class 'numpy.ndarray'>
ap 

Epoch 15/25
156/157 [============================>.] - ETA: 0s - loss: 0.0018 - binary_accuracy: 0.4226
Trying to save model @epoch=015 to /home/caleml/partial_experiments/exp_20190722_1416_TESTNB/model_60_015.h5
Couldn't save model, saving weights instead at /home/caleml/partial_experiments/exp_20190722_1416_TESTNB/weights_60_015.h5
type true <class 'numpy.ndarray'>, type pred <class 'numpy.ndarray'>
ap scores type <class 'numpy.ndarray'>
interval evaluation - epoch: 14 - mAP score: 0.837410
157/157 [==============================] - 135s 861ms/step - loss: 0.0019 - binary_accuracy: 0.4226
Epoch 16/25
156/157 [============================>.] - ETA: 1s - loss: 0.0015 - binary_accuracy: 0.4227
Trying to save model @epoch=016 to /home/caleml/partial_experiments/exp_20190722_1416_TESTNB/model_60_016.h5
Couldn't save model, saving weights instead at /home/caleml/partial_experiments/exp_20190722_1416_TESTNB/weights_60_016.h5
type true <class 'numpy.ndarray'>, type pred <class 'numpy.ndarray

## Dataset comparison

In [ ]:
data_dir = '/share/DEEPLEARNING/datasets/pascalvoc/VOCdevkit/VOC2007/'
data_generator_train = PascalVOCDataGenerator('trainval', data_dir)

In [ ]:
nico_data = data_generator_train.id_to_label
print(len(nico_data))
print(nico_data['000131'])

In [ ]:
trainval_path = '/share/DEEPLEARNING/datasets/pascalvoc/VOCdevkit/VOC2007/Annotations/annotations_multilabel_trainval.csv'

In [ ]:
laura_data = dict()
with open(trainval_path, 'r') as f_in:
    for line in f_in:
        parts = line.strip().split(',')
        laura_data[parts[0]] = [int(elt) for elt in parts[1:]]

In [ ]:
print(len(laura_data))
print(laura_data['000131'])

In [ ]:
for id_img, labels in nico_data.items():
    laura_labels = laura_data[id_img]
    converted = [l if l in [0, 1] else 0 for l in laura_labels]
    assert all([converted[i] == labels[i] for i in range(len(labels))])

## Launch.py surrogate